Resources used:
- https://python.useinstructor.com/hub/ollama/
- https://docs.pydantic.dev/latest/concepts/fields/

In [17]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI

# Define how the answer per player should look like
class PlayerResponse(BaseModel):
    player_id: int = Field(description="ID of the player")
    report_summary: str = Field(name="report_summary", description="Summary of the reports that have the same player id")
    
    
# We want to get a list of players
class ListPlayerResponse(BaseModel):
    list: List[PlayerResponse]


client = instructor.from_openai(
    OpenAI(
        # default port of ollama
        base_url="http://localhost:11434/v1",
        # need to specify this, otherwise we get an error from the library
        api_key="ollama"
    ),
    mode=instructor.Mode.JSON
)    


In [19]:
prompt = """You are an assistant in football (soccer) scouting, and provides answers to questions by using fact based information.
Use the following information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know. Dont just return the reports as they are but at least paraphrase them.

<context>
Player-ID: 1337, Report: Played as an attacking winger and used his speed and technique excellently. His dribbling led to several dangerous situations, although he again showed weaknesses defensively. He set up a goal with a precise cross and scored one himself after a solo run. His creativity and game vision were at a high level. At 23 years old, he still has a lot of potential to rise. \n\n Player-ID: 888, Report:  As a defensive midfielder, he again showed a disciplined and strong performance. Won many important duels in midfield and effectively stopped opponent attacks.His passing game was precise and he showed excellent vision. Offensively, he set no accents but focused on ball distribution. Showed weaknesses in fast counterattacks by the opponent. At 27 years old, he is at the peak of his footballing age and a reliable force in midfield.
</context>

<question>
I need a winger that has offensive skills and a defensive central player
</question>
"""

In [20]:
resp = client.chat.completions.create(
    model="mistral",
    messages=[{
        "role": "user",
        "content": prompt,
    }],
    response_model=ListPlayerResponse,
)

print(resp.model_dump_json(indent=2))

{
  "list": [
    {
      "player_id": 1337,
      "report_summary": "The winger played well offensively, using his speed, technique, and creativity to create dangerous situations. He displayed potential for growth but showed weaknesses defensively."
    },
    {
      "player_id": 888,
      "report_summary": "The defensive midfielder had a strong performance, effectively stopping opponent attacks with precision passing and excellent vision. Offensively, while he focused on ball distribution, he showed weaknesses in fast counterattacks."
    }
  ]
}
